Introduction to GPU Programming Summer School
========================

D. Quigley, University of Warwick

---

<table>
<tr>
    <td>
        <img src="https://images-na.ssl-images-amazon.com/images/I/71JLm2psPeL._SL1500_.jpg" width=300 height=300>
    </td>
    <td>
        <img src="https://images-na.ssl-images-amazon.com/images/I/61G6uH2aUxL._SL1200_.jpg" width=300 height=300>
    </td>
</tr>
    
<tr>
    <td width="500px">
        <center><h3>Intel 8-core central processing unit (CPU)</h3></center>
    </td>
    <td width="500px">
       <center><h3> Nvidia GeForce 1080 Graphics Processing Unit (GPU) with 2560 "CUDA cores"</h3></center>
    </td>
    
</tr>   
</table>

## GPU Pros?

* #### Parallel execution of thousands of threads vs a few dozen (maximum) on a CPU
* #### Huge energy efficiency in comparison to the same performance on a traditional CPU cluster

## GPU Cons?

* ####  Typically no more than 32GB of RAM per device, compared to 2-4GB per CPU core in traditional HPC clusters
* ####  Clock rate of around 1 GHz vs 3-4 GHz in CPUs, and less work done per clock 'tick'
* ####  Bandwidth between device memory and the compute units is fast, but not thousands of times faster than CPUs
* ####  Threads are grouped into *warps* which all execute instructions in lockstep
* ####  Code must be (re)written to explicitly make use of the GPU capabilities
* ####  Only a (growing) subset of your favourite langauge features can execute on a GPU
* ####  Not all computational tasks are suitable for GPU acceleration

---

## Requirements for GPU computing

* #### [A CUDA-capable Nvidia GPU](https://www.geforce.com/hardware/technology/cuda/supported-gpus)
* #### [CUDA Toolkit](https://developer.nvidia.com/cuda-toolkit)
* #### Cheap (£60) desktop cards fine for development and testing, but performance very limited
* #### High-end gaming cards (£600-£1,000) can be very powerful for CUDA but lack error correction/detection and features needed for remote monitoring and management in server environments. Poor double precision performance
* #### Tesla series aimed squarely at high performance computing market


| GPU card              |  Cores  | Single Precision TFLOPS | Double Precision TFLOPS   | GPU Memory Bandwidth GB/s  |
| ----------------------|:--------|:------------------------|:-------------------------:|---------------------------:|
| Tesla K20 (2012)        | 2496 | 3.52 | 1.18 |  208 |
| Tesla K40 ([Chiron](https://warwick.ac.uk/research/rtp/sc/hpc)) (2014)         | 2880 | 4.23 | 1.43 |  235 |
| Telsa K80 ([Tinis](https://warwick.ac.uk/research/rtp/sc/hpc)) (2015)   |  4992   | 5.59 | 1.87                      | 480                        |
| Tesla P100 ([Orac](https://warwick.ac.uk/research/rtp/sc/hpc)) (2017)   |  3584   | 8.07 | 4.36                       | 732                        |
| RTX 6000 ([Avon](https://warwick.ac.uk/research/rtp/sc/hpc)) (2020)     |  4608   | 16.03 | 0.51 | 672  |
| A100 (2020) |  6912   | 19.5 | 9.7 | 1,600  |
| GeForce 3080 (Gaming)|  10496   | 25.1 | 0.93                     | 936                        |

(Quoting [Wikipedia](https://en.wikipedia.org/wiki/Nvidia_Tesla) - TFLOPS shown are without Nvidia Boost)

---

## Programming model

* ####  CUDA is the (proprietary) programming model for exploiting Nvidia GPUs
* ####  Others exist (e.g. OpenCL) but arguably CUDA has become the de-facto standard and is much easier to learn
* ####  CUDA extends the C programming language with GPU features. CUDA Fortran also exists (and is now free via Nvidia HPC SDK)
* ####  Various third party tools for other languages - we will focus on [Numba](https://numba.pydata.org/) for Python as it appears closer in spirit to CUDA with compiled languages and so concepts are transferable

---

## CUDA and the [Scientific Computing Research Technology Platform (SCRTP)](https://warwick.ac.uk/research/rtp/sc)

* #### Managed desktop supports CUDA toolkit 10.1 for machines with GPU cards which support it
* #### CUDA 11 support on its way. Can be installed locally by users now on compatible machines
* #### Check support using `nvidia-smi` in the terminal
* #### Most desktop machines are supported. Some spare GeForce 1030 cards are available for machines which can accomodate them
* #### 16 GPU nodes each with 3x on Nvidia RTX 6000 hardware in Avon, and 1 OpenPower testbed with 4x Nvidia P100 cards in Orac. Available to all users via batch queues


## This course

* #### As hands-on as possible
* #### Using the SC RTP computers
* #### I will demonstrate codes snippets via Jupyter notebooks which you can download and tinker with as we go
* #### I am not showing you high quality code here - no exception handling etc
* #### We will use GPU nodes in the HPC clusters for some examples

---

# Tutorial 1: Preliminaries

### Sanity check of Python environment

All being well, you've reading this text inside a Jupyter notebook environment running on an SCRTP machine equipped with a CUDA-compatible GPU card and have launched the notebook from within an environment configured to support GPU computing. If in doubt, check the [connecting instructions](https://warwick.ac.uk/fac/sci/maths/research/events/2017-18/nonsymposium/gpu/connecting).

Let's run some simple checks to make sure you can execute code on a GPU from within this notebook. For today we'll be working with the Python interface to CUDA provided via the numba package.

Some terminology we need aleady:

**Host**        : The traditional computer in which our code is running on a CPU with access to host RAM.

**CUDA Device** : The GPU card consisting of its own RAM and computing cores (lots of them). 

In [ ]:
import platform          # So we can figure out where we're running
from numba import cuda   # Import python interface to CUDA

# Report where we're running
print("========================================================")
print("This notebook is running on ", platform.node())
print("========================================================")

# Test if CUDA is available. If so report on the devices present 
if cuda.is_available():  
    
    # List of CUDA capable devices in this system
    for device in cuda.list_devices():       
        print("Device ID : ", device.id, " : ", device.name.decode())         
    
else:
    print("There doesn't appear to be a CUDA capable device in this system")

Let's select the the most appropriate device and query its [compute capability](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities). CUDA devices have varying compute capability depending on the product range and when they were manufactured. Typically CUDA software will be written to support a minimum compute capability.

[Numba requires a compute capability of 2.0 or higher](https://numba.pydata.org/numba-doc/dev/cuda/overview.html) and the examples for this course require a minimum of 2.1, so we should check this.

In [ ]:
my_instance = cuda.select_device(0) # Create a device instance to work with based on device 0 

# The compute capability is stored as a tuple (major, minor) so we're good to go if...
if my_instance.compute_capability[0] >= 2 and my_instance.compute_capability[1] >= 1 :
    print("The selected device (",my_instance.name.decode(),") has a sufficient compute capability")
else:
    print("The selected device does not have a sufficient compute capability")

Normally we won't need to call ```cuda.select_device()```. The default context will be the fastest GPU in the machine. I'm only using this here to interrogate compute cabability.